<a href="https://colab.research.google.com/github/dolDolSee/TF2.0/blob/main/vgg_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

input_tensor = Input(shape=(224,224,3))
base_model = VGG16(input_tensor=input_tensor, include_top=True, weights="imagenet")
model = Model(inputs=input_tensor, outputs=base_model.output)
model.summary()

In [ ]:
import tensorflow as tf

class VGGnet(tf.keras.Model):
    def __init__(self, n_classes=10):
        super(VGGnet, self).__init__()
        
        self.conv_b1_layer1 = tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding="same", name="block1_conv1")
        self.conv_b1_layer2 = tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding="same", name="block1_conv2")
        self.b1_MaxPooling = tf.keras.layers.MaxPooling2D((2,2), strides=(2,2), name="block1_pool")
        
        self.conv_b2_layer1 = tf.keras.layers.Conv2D(128, (3,3), activation='relu', padding="same", name="block2_conv1")
        self.conv_b2_layer2 = tf.keras.layers.Conv2D(128, (3,3), activation='relu', padding="same", name="block2_conv2")
        self.b2_MaxPooling = tf.keras.layers.MaxPooling2D((2,2), strides=(2,2), name="block2_pool")
        
        self.conv_b3_layer1 = tf.keras.layers.Conv2D(256, (3,3), activation='relu', padding="same", name="block3_conv1")
        self.conv_b3_layer2 = tf.keras.layers.Conv2D(256, (3,3), activation='relu', padding="same", name="block3_conv2")
        self.conv_b3_layer3 = tf.keras.layers.Conv2D(256, (3,3), activation='relu', padding="same", name="block3_conv3")
        self.b3_MaxPooling = tf.keras.layers.MaxPooling2D((2,2), strides=(2,2), name="block3_pool")
        
        self.conv_b4_layer1 = tf.keras.layers.Conv2D(512, (3,3), activation='relu', padding="same", name="block4_conv1")
        self.conv_b4_layer2 = tf.keras.layers.Conv2D(512, (3,3), activation='relu', padding="same", name="block4_conv2")
        self.conv_b4_layer3 = tf.keras.layers.Conv2D(512, (3,3), activation='relu', padding="same", name="block4_conv3")
        self.b4_MaxPooling = tf.keras.layers.MaxPooling2D((2,2), strides=(2,2), name="block4_pool")
        
        self.conv_b5_layer1 = tf.keras.layers.Conv2D(512, (3,3), activation='relu', padding="same", name="block5_conv1")
        self.conv_b5_layer2 = tf.keras.layers.Conv2D(512, (3,3), activation='relu', padding="same", name="block5_conv2")
        self.conv_b5_layer3 = tf.keras.layers.Conv2D(512, (3,3), activation='relu', padding="same", name="block5_conv3")
        self.b5_MaxPooling = tf.keras.layers.MaxPooling2D((2,2), strides=(2,2), name="block5_pool")
        
        self.Gpooling = tf.keras.layers.GlobalAveragePooling2D()
        self.Dropout1 = tf.keras.layers.Dropout(0.5)
        self.Dense = tf.keras.layers.Dense(120, activation="relu")
        self.Dropout2 = tf.keras.layers.Dropout(0.5)
        self.output_layer = tf.keras.layers.Dense(n_classes, activation="softmax")
        
        
        
    def call(self, x):
        input = tf.reshape(x, [-1,128,128,3])
        x = self.conv_b1_layer1(input)
        x = self.conv_b1_layer2(x)
        x = self.b1_MaxPooling(x)
        x = self.conv_b2_layer1(x)
        x = self.conv_b2_layer2(x)
        x = self.b2_MaxPooling(x)
        x = self.conv_b3_layer1(x)
        x = self.conv_b3_layer2(x)
        x = self.conv_b3_layer3(x)
        x = self.b3_MaxPooling(x)
        x = self.conv_b4_layer1(x)
        x = self.conv_b4_layer2(x)
        x = self.conv_b4_layer3(x)
        x = self.b4_MaxPooling(x)
        x = self.conv_b5_layer1(x)
        x = self.conv_b5_layer2(x)
        x = self.conv_b5_layer3(x)
        x = self.b5_MaxPooling(x)
        x = self.Gpooling(x)
        x = self.Dropout1(x)
        x = self.Dense(x)
        x = self.Dropout2(x)
        x = self.output_layer(x)
        return x
    
model = VGGnet()
        
        
        
        

In [ ]:
IMAGE_SIZE = 128
BATCH_SIZE = 64

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd

import random as python_random
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import Sequence
import cv2
import sklearn

def zero_one_scaler(image):
    return image/255.0

def get_preprocessed_ohe(images, labels, pre_func=None):
    # preprocessing 함수가 입력되면 이를 이용하여 image array를 scaling 적용.
    if pre_func is not None:
        images = pre_func(images)
    # OHE 적용    
    oh_labels = to_categorical(labels)
    return images, oh_labels

# 학습/검증/테스트 데이터 세트에 전처리 및 OHE 적용한 뒤 반환 
def get_train_valid_test_set(train_images, train_labels, test_images, test_labels, valid_size=0.15, random_state=2021):
    # 학습 및 테스트 데이터 세트를  0 ~ 1사이값 float32로 변경 및 OHE 적용. 
    train_images, train_oh_labels = get_preprocessed_ohe(train_images, train_labels)
    test_images, test_oh_labels = get_preprocessed_ohe(test_images, test_labels)
    
    # 학습 데이터를 검증 데이터 세트로 다시 분리
    tr_images, val_images, tr_oh_labels, val_oh_labels = train_test_split(train_images, train_oh_labels, test_size=valid_size, random_state=random_state)
    
    return (tr_images, tr_oh_labels), (val_images, val_oh_labels), (test_images, test_oh_labels )

from tensorflow.keras.utils import Sequence
import cv2
import sklearn

# 입력 인자 images_array labels는 모두 numpy array로 들어옴. 
# 인자로 입력되는 images_array는 전체 32x32 image array임. 
class CIFAR_Dataset(Sequence):
    def __init__(self, images_array, labels, batch_size=BATCH_SIZE, augmentor=None, shuffle=False, pre_func=None):
        '''
        파라미터 설명
        images_array: 원본 32x32 만큼의 image 배열값. 
        labels: 해당 image의 label들
        batch_size: __getitem__(self, index) 호출 시 마다 가져올 데이터 batch 건수
        augmentor: albumentations 객체
        shuffle: 학습 데이터의 경우 epoch 종료시마다 데이터를 섞을지 여부
        '''
        # 객체 생성 인자로 들어온 값을 객체 내부 변수로 할당. 
        # 인자로 입력되는 images_array는 전체 32x32 image array임.
        self.images_array = images_array
        self.labels = labels
        self.batch_size = batch_size
        self.augmentor = augmentor
        self.pre_func = pre_func
        # train data의 경우 
        self.shuffle = shuffle
        if self.shuffle:
            # 객체 생성시에 한번 데이터를 섞음. 
            #self.on_epoch_end()
            pass
    
    # Sequence를 상속받은 Dataset은 batch_size 단위로 입력된 데이터를 처리함. 
    # __len__()은 전체 데이터 건수가 주어졌을 때 batch_size단위로 몇번 데이터를 반환하는지 나타남
    def __len__(self):
        # batch_size단위로 데이터를 몇번 가져와야하는지 계산하기 위해 전체 데이터 건수를 batch_size로 나누되, 정수로 정확히 나눠지지 않을 경우 1회를 더한다. 
        return int(np.ceil(len(self.labels) / self.batch_size))
    
    # batch_size 단위로 image_array, label_array 데이터를 가져와서 변환한 뒤 다시 반환함
    # 인자로 몇번째 batch 인지를 나타내는 index를 입력하면 해당 순서에 해당하는 batch_size 만큼의 데이타를 가공하여 반환
    # batch_size 갯수만큼 변환된 image_array와 label_array 반환. 
    def __getitem__(self, index):
        # index는 몇번째 batch인지를 나타냄. 
        # batch_size만큼 순차적으로 데이터를 가져오려면 array에서 index*self.batch_size:(index+1)*self.batch_size 만큼의 연속 데이터를 가져오면 됨
        # 32x32 image array를 self.batch_size만큼 가져옴. 
        images_fetch = self.images_array[index*self.batch_size:(index+1)*self.batch_size]
        if self.labels is not None:
            label_batch = self.labels[index*self.batch_size:(index+1)*self.batch_size]
        
        # 만일 객체 생성 인자로 albumentation으로 만든 augmentor가 주어진다면 아래와 같이 augmentor를 이용하여 image 변환
        # albumentations은 개별 image만 변환할 수 있으므로 batch_size만큼 할당된 image_name_batch를 한 건씩 iteration하면서 변환 수행. 
        # 변환된 image 배열값을 담을 image_batch 선언. image_batch 배열은 float32 로 설정. 
        image_batch = np.zeros((images_fetch.shape[0], IMAGE_SIZE, IMAGE_SIZE, 3), dtype='float32')
        
        # batch_size에 담긴 건수만큼 iteration 하면서 opencv image load -> image augmentation 변환(augmentor가 not None일 경우)-> image_batch에 담음. 
        for image_index in range(images_fetch.shape[0]):
            #image = cv2.cvtColor(cv2.imread(image_name_batch[image_index]), cv2.COLOR_BGR2RGB)
            # 원본 image를 IMAGE_SIZE x IMAGE_SIZE 크기로 변환
            image = cv2.resize(images_fetch[image_index], (IMAGE_SIZE, IMAGE_SIZE))
            # 만약 augmentor가 주어졌다면 이를 적용. 
            if self.augmentor is not None:
                image = self.augmentor(image=image)['image']
                
            # 만약 scaling 함수가 입력되었다면 이를 적용하여 scaling 수행. 
            if self.pre_func is not None:
                image = self.pre_func(image)
            
            # image_batch에 순차적으로 변환된 image를 담음.               
            image_batch[image_index] = image
        
        return image_batch, label_batch
    
    # epoch가 한번 수행이 완료 될 때마다 모델의 fit()에서 호출됨. 
    def on_epoch_end(self):
        if(self.shuffle):
            #print('epoch end')
            # 원본 image배열과 label를 쌍을 맞춰서 섞어준다. scikt learn의 utils.shuffle에서 해당 기능 제공
            self.images_array, self.labels = sklearn.utils.shuffle(self.images_array, self.labels)
        else:
            pass

In [ ]:
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
print(train_images.shape, train_labels.shape, test_images.shape, test_labels.shape)

(tr_images, tr_oh_labels), (val_images, val_oh_labels), (test_images, test_oh_labels) = \
    get_train_valid_test_set(train_images, train_labels, test_images, test_labels, valid_size=0.2, random_state=2021)
print(tr_images.shape, tr_oh_labels.shape, val_images.shape, val_oh_labels.shape, test_images.shape, test_oh_labels.shape)

In [ ]:
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_preprocess

tr_ds = CIFAR_Dataset(tr_images, tr_oh_labels, batch_size=BATCH_SIZE, augmentor=None, shuffle=True, pre_func=vgg_preprocess)
val_ds = CIFAR_Dataset(val_images, val_oh_labels, batch_size=BATCH_SIZE, augmentor=None, shuffle=False, pre_func=vgg_preprocess)

print(next(iter(tr_ds))[0].shape, next(iter(val_ds))[0].shape)
print(next(iter(tr_ds))[1].shape, next(iter(val_ds))[1].shape)
# 채널별 값 - mean = [103.939, 116.779, 123.68]
print(next(iter(tr_ds))[0][0])

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
model.compile(optimizer = tf.keras.optimizers.Adam(lr=0.0001), loss="categorical_crossentropy", metrics=['accuracy'])
rlr_cb = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, mode='min', verbose=1)
ely_cb = EarlyStopping(monitor='val_loss', patience=10, mode='min', verbose=1)
history = model.fit(tr_ds, epochs=10, validation_data=val_ds, callbacks=[rlr_cb, ely_cb])